In [288]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import geocoder
import requests
# transformers to impute missing data with sklearn:

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# to split the datasets
from sklearn.model_selection import train_test_split

In [289]:
df = pd.read_csv("greece_listings.csv")

In [290]:
df.head(20)

,location_name,location_region,res_date,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,deleted,deleted_at,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student,parking
0,Παλαιό Φάληρο,Αττική,2021-06-18,Διαμέρισμα,"Παλαιό Φάληρο,Γήπεδα",89000,1618.0,55.0,1973.0,Υπερυψωμένο,1.0,1.0,1,2022-04-05 00:00:00,Ανακαινισμένο,Δ,1,0,1,1,0,0,0,0,NaN
1,Αχαρνές,Αττική,2022-03-30,Μεζονέτα,"Αχαρνές,Λαθέα",150000,1531.0,98.0,2010.0,4ος,2.0,1.0,1,2022-05-19 00:00:00,Νεόδμητο,Γ,0,0,0,0,0,1,0,0,Κλειστό parking
2,Παγκράτι,Αττική,2022-02-25,Διαμέρισμα,"Αθήνα,Παγκράτι",400000,3175.0,126.0,1990.0,3ος,3.0,1.0,1,2022-06-10 00:00:00,Άριστη,Δ,1,0,1,1,0,1,0,0,NaN
3,Αθήνα,Αττική,2022-03-21,Διαμέρισμα,"Αθήνα,Άγιος Παντελεήμονας",80000,941.0,85.0,1957.0,5ος,2.0,1.0,1,2022-03-23 00:00:00,Καλή,Ζ,1,0,0,1,0,0,0,1,NaN
4,Ίλιον,Αττική,2022-01-11,Διαμέρισμα,"Αθήνα,Άγιος Ελευθέριος",100000,2083.0,48.0,1996.0,1ος,1.0,1.0,1,2022-04-14 00:00:00,Καλή,Δ,1,0,0,1,0,0,0,0,NaN
5,Αθήνα,Αττική,2019-12-04,Διαμέρισμα,"Αθήνα,Πλατεία Αμερικής",133000,1385.0,96.0,1964.0,3ος,2.0,1.0,1,2022-03-30 00:00:00,Ανακαινισμένο,Εκρεμμεί,1,0,0,1,0,0,0,0,NaN
6,Γαλάτσι,Αττική,2020-09-11,Διαμέρισμα,"Αθήνα,Άνω Κυψέλη",35000,522.0,67.0,1970.0,Ημιυπόγειο,2.0,1.0,1,2022-03-17 00:00:00,Καλή,Ε,0,0,0,0,0,0,0,0,NaN
7,Λαγκάδας,Θεσσαλονίκη,2021-11-27,Μονοκατοικία,Περιβολάκι,89000,318.0,280.0,2012.0,Ισόγειο,5.0,2.0,1,2022-05-29 00:00:00,Καλή,Β,0,0,0,1,0,0,0,0,Κλειστό parking
8,Αθήνα,Αττική,2022-02-14,Διαμέρισμα,"Αθήνα,Προφήτης Ηλίας",130000,1733.0,75.0,1972.0,1ος,2.0,2.0,1,2022-06-10 00:00:00,Ανακαινισμένο,Δ,0,0,1,1,0,0,0,0,NaN
9,Δάφνη,Αττική,2022-04-11,Κτίριο,"Αθήνα,Κυνοσάργους",450000,1331.0,338.0,2002.0,"Υπόγειο,Ισόγειο,1ος,2ος",3.0,2.0,1,2022-04-22 00:00:00,Καλή,Εκρεμμεί,0,1,1,1,1,1,0,0,"Κλειστό parking, Υπόγειο parking"


In [258]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   location_name      20000 non-null  object 
 1   location_region    20000 non-null  object 
 2   res_date           20000 non-null  object 
 3   res_type           20000 non-null  object 
 4   res_address        20000 non-null  object 
 5   res_price          20000 non-null  int64  
 6   res_price_sqr      19998 non-null  float64
 7   res_sqr            19999 non-null  float64
 8   construction_year  19214 non-null  float64
 9   levels             19992 non-null  object 
 10  bedrooms           18935 non-null  float64
 11  bathrooms          18214 non-null  float64
 12  deleted            20000 non-null  int64  
 13  deleted_at         16885 non-null  object 
 14  status             18195 non-null  object 
 15  energyclass        18186 non-null  object 
 16  auto_heating       200

In [259]:
pd.set_option('display.max_columns', None)

In [260]:
df

,location_name,location_region,res_date,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,deleted,deleted_at,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student,parking
0,Παλαιό Φάληρο,Αττική,2021-06-18,Διαμέρισμα,"Παλαιό Φάληρο,Γήπεδα",89000,1618.0,55.0,1973.0,Υπερυψωμένο,1.0,1.0,1,2022-04-05 00:00:00,Ανακαινισμένο,Δ,1,0,1,1,0,0,0,0,NaN
1,Αχαρνές,Αττική,2022-03-30,Μεζονέτα,"Αχαρνές,Λαθέα",150000,1531.0,98.0,2010.0,4ος,2.0,1.0,1,2022-05-19 00:00:00,Νεόδμητο,Γ,0,0,0,0,0,1,0,0,Κλειστό parking
2,Παγκράτι,Αττική,2022-02-25,Διαμέρισμα,"Αθήνα,Παγκράτι",400000,3175.0,126.0,1990.0,3ος,3.0,1.0,1,2022-06-10 00:00:00,Άριστη,Δ,1,0,1,1,0,1,0,0,NaN
3,Αθήνα,Αττική,2022-03-21,Διαμέρισμα,"Αθήνα,Άγιος Παντελεήμονας",80000,941.0,85.0,1957.0,5ος,2.0,1.0,1,2022-03-23 00:00:00,Καλή,Ζ,1,0,0,1,0,0,0,1,NaN
4,Ίλιον,Αττική,2022-01-11,Διαμέρισμα,"Αθήνα,Άγιος Ελευθέριος",100000,2083.0,48.0,1996.0,1ος,1.0,1.0,1,2022-04-14 00:00:00,Καλή,Δ,1,0,0,1,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Βάρη - Βούλα - Βουλιαγμένη,Αττική,2022-04-10,Διαμέρισμα,"Βούλα,Κάτω Βούλα",145000,2900.0,50.0,NaN,Ισόγειο,1.0,1.0,1,2022-05-02 00:00:00,NaN,Εκρεμμεί,0,0,0,0,0,0,0,0,NaN
19996,Αθήνα,Αττική,2022-05-24,Διαμέρισμα,"Αθήνα,Κολιάτσου",92000,1022.0,90.0,1962.0,2ος,3.0,1.0,0,NaN,Ανακαινισμένο,Εκρεμμεί,0,0,0,1,0,0,0,0,NaN
19997,Νίκαια,Αττική,2021-11-24,Κτίριο,"Κορυδαλλός,Όριο Αγίας Βαρβάρας",200000,939.0,213.0,1985.0,Ισόγειο,6.0,2.0,0,NaN,NaN,NaN,1,1,1,0,0,0,0,1,NaN
19998,Νέα Ιωνία,Αττική,2022-03-22,Μεζονέτα,"Ηράκλειο,Πράσινος Λόφος",355000,2840.0,125.0,2022.0,Ισόγειο,4.0,1.0,1,2022-03-25 00:00:00,Νεόδμητο,Α+,1,1,0,1,0,0,0,0,NaN


In [261]:
missing_data=df.isna()

In [262]:
missing_data_num= missing_data.sum()

In [263]:
df.isna().mean().round(4) * 100

location_name         0.00
location_region       0.00
res_date              0.00
res_type              0.00
res_address           0.00
res_price             0.00
res_price_sqr         0.01
res_sqr               0.00
construction_year     3.93
levels                0.04
bedrooms              5.32
bathrooms             8.93
deleted               0.00
deleted_at           15.58
status                9.02
energyclass           9.07
auto_heating          0.00
solar                 0.00
cooling               0.00
safe_door             0.00
gas                   0.00
fireplace             0.00
furniture             0.00
student               0.00
parking              83.42
dtype: float64

In [264]:
df[df.bedrooms.isna()]

,location_name,location_region,res_date,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,deleted,deleted_at,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student,parking
18,Αχαρνές,Αττική,2022-03-02,Μεζονέτα,"Κηφισιά,Κέντρο",1450000,2984.0,486.0,NaN,"Ισόγειο,1ος,2ος,3ος",NaN,NaN,1,2022-04-16 00:00:00,Νεόδμητο,Εκρεμμεί,0,0,0,0,0,0,0,0,NaN
30,Αχαρνές,Αττική,2022-03-03,Μεζονέτα,Θρακομακεδόνες,700000,2333.0,300.0,1999.0,Υπόγειο,NaN,NaN,1,2022-03-17 00:00:00,Καλή,Γ,0,0,0,0,0,0,0,0,NaN
68,Κρωπία,Αττική,2022-03-19,Οικία,"Καλύβια Θορικού,Λαγονήσι (Παραλία)",120000,1600.0,75.0,1970.0,Ισόγειο,NaN,NaN,1,2022-06-10 00:00:00,Χρήζει ανακαίνισης,Μη αποδοτικό,0,0,0,0,0,0,0,0,NaN
77,Πειραιάς,Αττική,2022-01-19,Κτίριο,"Πειραιάς,Ταμπούρια",300000,857.0,350.0,1967.0,"Ισόγειο,1ος,2ος,3ος",NaN,NaN,1,2022-06-10 00:00:00,Χρήζει ανακαίνισης,Η,0,0,0,0,0,0,0,0,NaN
85,Αθήνα,Αττική,2021-07-12,Κτίριο,"Αθήνα,Εξάρχεια",1000000,1786.0,560.0,1923.0,"Ισόγειο,1ος,2ος",NaN,6.0,1,2022-06-07 00:00:00,Άλλη κατάσταση,Η,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19914,Βριλήσσια,Αττική,2021-04-26,Κτίριο,"Βριλήσσια,Άνω Βριλήσσια",650000,1467.0,443.0,1992.0,1ος,NaN,NaN,1,2022-03-18 00:00:00,Καλή,Β,0,0,1,1,0,0,0,0,Ανοιχτό parking
19918,Αχαρνές,Αττική,2021-07-07,Μονοκατοικία,Ζεφύρι,60000,938.0,64.0,1965.0,Ισόγειο,NaN,NaN,0,NaN,Χρήζει ανακαίνισης,Ζ,0,0,0,0,0,0,0,0,NaN
19937,Βύρωνας,Αττική,2021-11-11,Διαμέρισμα,Βύρωνας,185000,1762.0,105.0,1996.0,6ος,NaN,NaN,0,NaN,Καλή,Μη αποδοτικό,0,0,0,0,0,0,0,0,NaN
19959,Μαρούσι,Αττική,2021-06-08,Διαμέρισμα,"Μαρούσι,Κέντρο",85000,2931.0,29.0,NaN,1ος,NaN,NaN,1,2022-05-03 00:00:00,NaN,Ε,0,0,0,0,0,0,0,0,NaN


In [265]:
df[df.duplicated()]

,location_name,location_region,res_date,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,deleted,deleted_at,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student,parking
9017,Λαγκάδας,Θεσσαλονίκη,2021-10-19,Μεζονέτα,Χορτιάτης,130000,1083.0,120.0,2022.0,1ος,3.0,1.0,0,NaN,Νεόδμητο,Α,0,0,0,0,1,0,0,0,NaN
15789,Άλιμος,Αττική,2022-04-25,Διαμέρισμα,"Άγιος Δημήτριος Αττικής,Μοναστήριο",240000,2824.0,85.0,2021.0,2ος,2.0,1.0,1,2022-05-07 00:00:00,Υπο κατασκευή,Β,0,0,0,1,0,1,0,0,NaN


In [266]:
df.describe()

,res_price,res_price_sqr,res_sqr,construction_year,bedrooms,bathrooms,deleted,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student
count,2.000000e+04,19998.000000,19999.000000,19214.000000,18935.000000,18214.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,3.671051e+05,2423.929493,169.144757,1990.505413,2.582836,1.47996,0.844250,0.525400,0.276750,0.350750,0.552400,0.147300,0.320650,0.071250,0.07515
std,5.983191e+05,2127.543954,2300.216048,22.139631,1.708345,1.15161,0.362627,0.499367,0.447403,0.477217,0.497259,0.354414,0.466738,0.257248,0.26364
min,4.300000e+01,1.000000,10.000000,1900.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.200000e+05,1405.250000,72.000000,1972.000000,2.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,2.200000e+05,2114.000000,99.000000,1989.000000,2.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000
75%,3.950000e+05,3028.000000,150.000000,2010.000000,3.000000,2.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.00000
max,2.500000e+07,208333.000000,320000.000000,2022.000000,51.000000,26.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [267]:
df.drop(['deleted', 'res_date','deleted_at','student'], axis=1)

,location_name,location_region,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,parking
0,Παλαιό Φάληρο,Αττική,Διαμέρισμα,"Παλαιό Φάληρο,Γήπεδα",89000,1618.0,55.0,1973.0,Υπερυψωμένο,1.0,1.0,Ανακαινισμένο,Δ,1,0,1,1,0,0,0,NaN
1,Αχαρνές,Αττική,Μεζονέτα,"Αχαρνές,Λαθέα",150000,1531.0,98.0,2010.0,4ος,2.0,1.0,Νεόδμητο,Γ,0,0,0,0,0,1,0,Κλειστό parking
2,Παγκράτι,Αττική,Διαμέρισμα,"Αθήνα,Παγκράτι",400000,3175.0,126.0,1990.0,3ος,3.0,1.0,Άριστη,Δ,1,0,1,1,0,1,0,NaN
3,Αθήνα,Αττική,Διαμέρισμα,"Αθήνα,Άγιος Παντελεήμονας",80000,941.0,85.0,1957.0,5ος,2.0,1.0,Καλή,Ζ,1,0,0,1,0,0,0,NaN
4,Ίλιον,Αττική,Διαμέρισμα,"Αθήνα,Άγιος Ελευθέριος",100000,2083.0,48.0,1996.0,1ος,1.0,1.0,Καλή,Δ,1,0,0,1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Βάρη - Βούλα - Βουλιαγμένη,Αττική,Διαμέρισμα,"Βούλα,Κάτω Βούλα",145000,2900.0,50.0,NaN,Ισόγειο,1.0,1.0,NaN,Εκρεμμεί,0,0,0,0,0,0,0,NaN
19996,Αθήνα,Αττική,Διαμέρισμα,"Αθήνα,Κολιάτσου",92000,1022.0,90.0,1962.0,2ος,3.0,1.0,Ανακαινισμένο,Εκρεμμεί,0,0,0,1,0,0,0,NaN
19997,Νίκαια,Αττική,Κτίριο,"Κορυδαλλός,Όριο Αγίας Βαρβάρας",200000,939.0,213.0,1985.0,Ισόγειο,6.0,2.0,NaN,NaN,1,1,1,0,0,0,0,NaN
19998,Νέα Ιωνία,Αττική,Μεζονέτα,"Ηράκλειο,Πράσινος Λόφος",355000,2840.0,125.0,2022.0,Ισόγειο,4.0,1.0,Νεόδμητο,Α+,1,1,0,1,0,0,0,NaN


In [268]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("res_price", axis=1),  # just the features
    df["res_price"],  # the target
    test_size=0.3,  # the percentage of obs in the test set
    random_state=0,  # for reproducibility
)

X_train.shape, X_test.shape

((14000, 24), (6000, 24))

In [269]:
# let's check the misssing data again
X_train.isnull().mean()

location_name        0.000000
location_region      0.000000
res_date             0.000000
res_type             0.000000
res_address          0.000000
res_price_sqr        0.000143
res_sqr              0.000071
construction_year    0.039857
levels               0.000357
bedrooms             0.054929
bathrooms            0.091143
deleted              0.000000
deleted_at           0.158500
status               0.089857
energyclass          0.091143
auto_heating         0.000000
solar                0.000000
cooling              0.000000
safe_door            0.000000
gas                  0.000000
fireplace            0.000000
furniture            0.000000
student              0.000000
parking              0.833000
dtype: float64

In [270]:
# first we need to make lists, indicating which features
# will be imputed with each method

features_numeric = ["res_price_sqr", "res_sqr", "construction_year","bedrooms","bathrooms",]
features_categoric = ["status", "energyclass","parking"]

# then we put the features list and the transformers to
# the column transformer

preprocessor = ColumnTransformer(
    transformers=[
        ("imputer_numeric", SimpleImputer(strategy="mean"), features_numeric),
        (
            "imputer_categoric",
            SimpleImputer(strategy="most_frequent"),
            features_categoric,
        ),
    ]
)

In [271]:
# set output to be a dataframe

preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('imputer_numeric', SimpleImputer(),
                                 ['res_price_sqr', 'res_sqr',
                                  'construction_year', 'bedrooms',
                                  'bathrooms']),
                                ('imputer_categoric',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['status', 'energyclass', 'parking'])])

In [272]:
# now we fit the preprocessor
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('imputer_numeric', SimpleImputer(),
                                 ['res_price_sqr', 'res_sqr',
                                  'construction_year', 'bedrooms',
                                  'bathrooms']),
                                ('imputer_categoric',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['status', 'energyclass', 'parking'])])

In [273]:
### and now we can impute the data.
# Remember that it returs a numpy array.

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [274]:
#X_train.shape
X_train.head(20)

,imputer_numeric__res_price_sqr,imputer_numeric__res_sqr,imputer_numeric__construction_year,imputer_numeric__bedrooms,imputer_numeric__bathrooms,imputer_categoric__status,imputer_categoric__energyclass,imputer_categoric__parking
8226,1579.0,57.0,2004.000000,1.0,1.000000,Άριστη,Μη αποδοτικό,Κλειστό parking
9398,5385.0,325.0,1975.000000,3.0,1.000000,Καλή,Η,Κλειστό parking
11646,1029.0,68.0,1978.000000,2.0,1.000000,Καλή,Ε,Κλειστό parking
12193,1077.0,65.0,1955.000000,1.0,1.000000,Άριστη,Ε,Κλειστό parking
18601,2500.0,440.0,1990.000000,4.0,1.000000,Άριστη,Γ,Κλειστό parking
12427,1300.0,100.0,1970.000000,3.0,1.479802,Καλή,Ζ,Κλειστό parking
13257,1458.0,60.0,1985.000000,1.0,1.000000,Άλλη κατάσταση,Β,Κλειστό parking
482,1857.0,105.0,1974.000000,2.0,1.000000,Ανακαινισμένο,Ε,Κλειστό parking
11304,2700.0,100.0,1995.000000,2.0,2.000000,Καλή,Δ,Κλειστό parking
19981,4815.0,135.0,2021.000000,3.0,2.000000,Νεόδμητο,Α+,Κλειστό parking


In [284]:
df2=df.combine_first(df1)

In [286]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   imputer_numeric__res_price_sqr      20000 non-null  float64
 1   imputer_numeric__res_sqr            20000 non-null  float64
 2   imputer_numeric__construction_year  20000 non-null  float64
 3   imputer_numeric__bedrooms           20000 non-null  float64
 4   imputer_numeric__bathrooms          20000 non-null  float64
 5   imputer_categoric__status           20000 non-null  object 
 6   imputer_categoric__energyclass      20000 non-null  object 
 7   imputer_categoric__parking          20000 non-null  object 
dtypes: float64(5), object(3)
memory usage: 1.4+ MB


In [287]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   location_name      20000 non-null  object 
 1   location_region    20000 non-null  object 
 2   res_date           20000 non-null  object 
 3   res_type           20000 non-null  object 
 4   res_address        20000 non-null  object 
 5   res_price          20000 non-null  int64  
 6   res_price_sqr      19998 non-null  float64
 7   res_sqr            19999 non-null  float64
 8   construction_year  19214 non-null  float64
 9   levels             19992 non-null  object 
 10  bedrooms           18935 non-null  float64
 11  bathrooms          18214 non-null  float64
 12  deleted            20000 non-null  int64  
 13  deleted_at         16885 non-null  object 
 14  status             18195 non-null  object 
 15  energyclass        18186 non-null  object 
 16  auto_heating       200

In [229]:
#X_test.shape
X_test.head(20)

,imputer_numeric__res_price_sqr,imputer_numeric__res_sqr,imputer_numeric__construction_year,imputer_numeric__bedrooms,imputer_numeric__bathrooms,imputer_categoric__status,imputer_categoric__energyclass,imputer_categoric__parking
19134,1400.0,50.0,1970.0,1.000000,1.000000,Καλή,Ζ,Κλειστό parking
4981,1140.0,57.0,1964.0,1.000000,1.000000,Χρήζει ανακαίνισης,Ζ,Κλειστό parking
16643,1680.0,250.0,1985.0,5.000000,3.000000,Καλή,Δ,Κλειστό parking
19117,5556.0,198.0,2022.0,1.000000,5.000000,Νεόδμητο,Α,Κλειστό parking
5306,1220.0,82.0,1978.0,2.000000,1.000000,Άλλη κατάσταση,Η,Κλειστό parking
230,1944.0,72.0,1988.0,2.000000,1.000000,Καλή,Δ,Κλειστό parking
3148,2723.0,101.0,2021.0,3.000000,1.000000,Νεόδμητο,Α,Ανοιχτό parking
11525,2759.0,87.0,2010.0,2.000000,1.000000,Άριστη,Εκρεμμεί,Κλειστό parking
13672,1928.0,83.0,1977.0,2.000000,1.000000,Ανακαινισμένο,Ζ,Κλειστό parking
1624,1279.0,86.0,1980.0,2.000000,1.000000,Ανακαινισμένο,Δ,Κλειστό parking


In [279]:
X1=X_train.sort_index(axis=0)

In [280]:
X2=X_test.sort_index(axis=0)

In [281]:
df1 = pd.concat([X1, X2], axis="rows").sort_index(axis=0)


In [282]:
df1.shape

(20000, 8)

In [283]:
df1.head(20)

,imputer_numeric__res_price_sqr,imputer_numeric__res_sqr,imputer_numeric__construction_year,imputer_numeric__bedrooms,imputer_numeric__bathrooms,imputer_categoric__status,imputer_categoric__energyclass,imputer_categoric__parking
0,1618.0,55.0,1973.000000,1.000000,1.000000,Ανακαινισμένο,Δ,Κλειστό parking
1,1531.0,98.0,2010.000000,2.000000,1.000000,Νεόδμητο,Γ,Κλειστό parking
2,3175.0,126.0,1990.000000,3.000000,1.000000,Άριστη,Δ,Κλειστό parking
3,941.0,85.0,1957.000000,2.000000,1.000000,Καλή,Ζ,Κλειστό parking
4,2083.0,48.0,1996.000000,1.000000,1.000000,Καλή,Δ,Κλειστό parking
5,1385.0,96.0,1964.000000,2.000000,1.000000,Ανακαινισμένο,Εκρεμμεί,Κλειστό parking
6,522.0,67.0,1970.000000,2.000000,1.000000,Καλή,Ε,Κλειστό parking
7,318.0,280.0,2012.000000,5.000000,2.000000,Καλή,Β,Κλειστό parking
8,1733.0,75.0,1972.000000,2.000000,2.000000,Ανακαινισμένο,Δ,Κλειστό parking
9,1331.0,338.0,2002.000000,3.000000,2.000000,Καλή,Εκρεμμεί,"Κλειστό parking, Υπόγειο parking"


In [241]:
X1.head(20)

,imputer_numeric__res_price_sqr,imputer_numeric__res_sqr,imputer_numeric__construction_year,imputer_numeric__bedrooms,imputer_numeric__bathrooms,imputer_categoric__status,imputer_categoric__energyclass,imputer_categoric__parking
0,1618.0,55.0,1973.000000,1.0,1.0,Ανακαινισμένο,Δ,Κλειστό parking
1,1531.0,98.0,2010.000000,2.0,1.0,Νεόδμητο,Γ,Κλειστό parking
2,3175.0,126.0,1990.000000,3.0,1.0,Άριστη,Δ,Κλειστό parking
3,941.0,85.0,1957.000000,2.0,1.0,Καλή,Ζ,Κλειστό parking
4,2083.0,48.0,1996.000000,1.0,1.0,Καλή,Δ,Κλειστό parking
5,1385.0,96.0,1964.000000,2.0,1.0,Ανακαινισμένο,Εκρεμμεί,Κλειστό parking
6,522.0,67.0,1970.000000,2.0,1.0,Καλή,Ε,Κλειστό parking
7,318.0,280.0,2012.000000,5.0,2.0,Καλή,Β,Κλειστό parking
10,1250.0,88.0,1982.000000,2.0,1.0,Άριστη,Ε,Κλειστό parking
11,2558.0,86.0,2002.000000,2.0,1.0,Άριστη,Γ,Parking πιλωτής


In [239]:
X2.head(20)

,imputer_numeric__res_price_sqr,imputer_numeric__res_sqr,imputer_numeric__construction_year,imputer_numeric__bedrooms,imputer_numeric__bathrooms,imputer_categoric__status,imputer_categoric__energyclass,imputer_categoric__parking
8,1733.0,75.0,1972.000000,2.000000,2.000000,Ανακαινισμένο,Δ,Κλειστό parking
9,1331.0,338.0,2002.000000,3.000000,2.000000,Καλή,Εκρεμμεί,"Κλειστό parking, Υπόγειο parking"
12,2737.0,95.0,1985.000000,2.000000,1.000000,Άριστη,Γ,Κλειστό parking
16,1091.0,110.0,1925.000000,2.000000,1.000000,Καλή,Δ,Κλειστό parking
18,2984.0,486.0,1990.644696,2.568967,1.479802,Νεόδμητο,Εκρεμμεί,Κλειστό parking
31,3465.0,101.0,2021.000000,3.000000,2.000000,Νεόδμητο,Β+,Κλειστό parking
33,1889.0,81.0,1971.000000,2.000000,1.000000,Ανακαινισμένο,Δ,Κλειστό parking
35,1231.0,52.0,1971.000000,1.000000,1.000000,Καλή,Ε,Κλειστό parking
37,2783.0,115.0,2008.000000,3.000000,1.000000,Καλή,Γ,Parking πιλωτής
44,2421.0,95.0,1981.000000,2.000000,1.479802,Ανακαινισμένο,Εκρεμμεί,Κλειστό parking


In [223]:
df1=pd.concat([X_test, X_test])

In [225]:
df1.shape

(12000, 8)

In [ ]:
energyclass=['Δ', 'Γ', 'Ζ', 'Εκρεμμεί', 'Ε', 'Β', nan, 'Α', 'Μη αποδοτικό',
       'Β+', 'Η', 'Α+', 'Εξαιρείται']

In [30]:

df
with requests.Session() as session:
    for address in df["location_name"]:
        coo = geocoder.osm(address, session=session)
        if coo:
            df.loc[df["location_name"] == address, "Long"] = round(coo.osm["x"], 2)
            df.loc[df["location_name"] == address, "Lat"] = round(coo.osm["y"], 2)


In [156]:
df


,location_name,location_region,res_date,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,deleted,deleted_at,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student,parking
0,Παλαιό Φάληρο,Αττική,2021-06-18,Διαμέρισμα,"Παλαιό Φάληρο,Γήπεδα",89000,1618.0,55.0,1973.0,Υπερυψωμένο,1.0,1.0,1,2022-04-05 00:00:00,Ανακαινισμένο,Δ,1,0,1,1,0,0,0,0,NaN
1,Αχαρνές,Αττική,2022-03-30,Μεζονέτα,"Αχαρνές,Λαθέα",150000,1531.0,98.0,2010.0,4ος,2.0,1.0,1,2022-05-19 00:00:00,Νεόδμητο,Γ,0,0,0,0,0,1,0,0,Κλειστό parking
2,Παγκράτι,Αττική,2022-02-25,Διαμέρισμα,"Αθήνα,Παγκράτι",400000,3175.0,126.0,1990.0,3ος,3.0,1.0,1,2022-06-10 00:00:00,Άριστη,Δ,1,0,1,1,0,1,0,0,NaN
3,Αθήνα,Αττική,2022-03-21,Διαμέρισμα,"Αθήνα,Άγιος Παντελεήμονας",80000,941.0,85.0,1957.0,5ος,2.0,1.0,1,2022-03-23 00:00:00,Καλή,Ζ,1,0,0,1,0,0,0,1,NaN
4,Ίλιον,Αττική,2022-01-11,Διαμέρισμα,"Αθήνα,Άγιος Ελευθέριος",100000,2083.0,48.0,1996.0,1ος,1.0,1.0,1,2022-04-14 00:00:00,Καλή,Δ,1,0,0,1,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Βάρη - Βούλα - Βουλιαγμένη,Αττική,2022-04-10,Διαμέρισμα,"Βούλα,Κάτω Βούλα",145000,2900.0,50.0,NaN,Ισόγειο,1.0,1.0,1,2022-05-02 00:00:00,NaN,Εκρεμμεί,0,0,0,0,0,0,0,0,NaN
19996,Αθήνα,Αττική,2022-05-24,Διαμέρισμα,"Αθήνα,Κολιάτσου",92000,1022.0,90.0,1962.0,2ος,3.0,1.0,0,NaN,Ανακαινισμένο,Εκρεμμεί,0,0,0,1,0,0,0,0,NaN
19997,Νίκαια,Αττική,2021-11-24,Κτίριο,"Κορυδαλλός,Όριο Αγίας Βαρβάρας",200000,939.0,213.0,1985.0,Ισόγειο,6.0,2.0,0,NaN,NaN,NaN,1,1,1,0,0,0,0,1,NaN
19998,Νέα Ιωνία,Αττική,2022-03-22,Μεζονέτα,"Ηράκλειο,Πράσινος Λόφος",355000,2840.0,125.0,2022.0,Ισόγειο,4.0,1.0,1,2022-03-25 00:00:00,Νεόδμητο,Α+,1,1,0,1,0,0,0,0,NaN


In [162]:
df

,location_name,location_region,res_date,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,deleted,deleted_at,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student,parking
0,Παλαιό Φάληρο,Αττική,2021-06-18,Διαμέρισμα,"Παλαιό Φάληρο,Γήπεδα",89000,1618.0,55.0,1973.0,0.0,1.0,1.0,1,2022-04-05 00:00:00,Ανακαινισμένο,Δ,1,0,1,1,0,0,0,0,NaN
1,Αχαρνές,Αττική,2022-03-30,Μεζονέτα,"Αχαρνές,Λαθέα",150000,1531.0,98.0,2010.0,4.0,2.0,1.0,1,2022-05-19 00:00:00,Νεόδμητο,Γ,0,0,0,0,0,1,0,0,Κλειστό parking
2,Παγκράτι,Αττική,2022-02-25,Διαμέρισμα,"Αθήνα,Παγκράτι",400000,3175.0,126.0,1990.0,3.0,3.0,1.0,1,2022-06-10 00:00:00,Άριστη,Δ,1,0,1,1,0,1,0,0,NaN
3,Αθήνα,Αττική,2022-03-21,Διαμέρισμα,"Αθήνα,Άγιος Παντελεήμονας",80000,941.0,85.0,1957.0,5.0,2.0,1.0,1,2022-03-23 00:00:00,Καλή,Ζ,1,0,0,1,0,0,0,1,NaN
4,Ίλιον,Αττική,2022-01-11,Διαμέρισμα,"Αθήνα,Άγιος Ελευθέριος",100000,2083.0,48.0,1996.0,1.0,1.0,1.0,1,2022-04-14 00:00:00,Καλή,Δ,1,0,0,1,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Βάρη - Βούλα - Βουλιαγμένη,Αττική,2022-04-10,Διαμέρισμα,"Βούλα,Κάτω Βούλα",145000,2900.0,50.0,NaN,0.0,1.0,1.0,1,2022-05-02 00:00:00,NaN,Εκρεμμεί,0,0,0,0,0,0,0,0,NaN
19996,Αθήνα,Αττική,2022-05-24,Διαμέρισμα,"Αθήνα,Κολιάτσου",92000,1022.0,90.0,1962.0,2.0,3.0,1.0,0,NaN,Ανακαινισμένο,Εκρεμμεί,0,0,0,1,0,0,0,0,NaN
19997,Νίκαια,Αττική,2021-11-24,Κτίριο,"Κορυδαλλός,Όριο Αγίας Βαρβάρας",200000,939.0,213.0,1985.0,0.0,6.0,2.0,0,NaN,NaN,NaN,1,1,1,0,0,0,0,1,NaN
19998,Νέα Ιωνία,Αττική,2022-03-22,Μεζονέτα,"Ηράκλειο,Πράσινος Λόφος",355000,2840.0,125.0,2022.0,0.0,4.0,1.0,1,2022-03-25 00:00:00,Νεόδμητο,Α+,1,1,0,1,0,0,0,0,NaN


In [ ]:
#energy class
A+
A
B+
B
Γ
Δ
Ε
Ζ
Η
Μη αποδοτικό
Εκρεμμεί
Εξαιρείται

#parking
array(['Κλειστό parking', 'Κλειστό parking, Υπόγειο parking',
       'Parking πιλωτής', 'Υπόγειο parking', 'Ανοιχτό parking',
       'Ανοιχτό parking, Parking πιλωτής',
       'Ανοιχτό parking, Κλειστό parking',
       'Κλειστό parking, Parking πιλωτής',
       'Parking πιλωτής, Υπόγειο parking',
       'Ανοιχτό parking, Υπόγειο parking',
       'Ανοιχτό parking, Κλειστό parking, Parking πιλωτής, Υπόγειο parking'],
      dtype=object)
assuming Ανοιχτό parking is no private parking


In [53]:

df.loc[df['energyclass']=='Α+','energyclass']=1.0
df.loc[df['energyclass']=='Α','energyclass']=2.0
df.loc[df['energyclass']=='Β+','energyclass']=3.0
df.loc[df['energyclass']=='Β','energyclass']=4.0
df.loc[df['energyclass']=='Γ','energyclass']=5.0
df.loc[df['energyclass']=='Δ','energyclass']=6.0
df.loc[df['energyclass']=='Ε','energyclass']=7.0
df.loc[df['energyclass']=='Ζ','energyclass']=8.0
df.loc[df['energyclass']=='Η','energyclass']=9.0
df.loc[df['energyclass']=='Μη αποδοτικό','energyclass']=10.0
df.loc[df['energyclass']=='Εκρεμμεί','energyclass']=11.0
df.loc[df['energyclass']=='Ε','energyclass']=12.0
df.loc[df['energyclass']=='Εξαιρείται','energyclass']=13.0

df[['energyclass']] = df[['energyclass']].apply(pd.to_numeric)



In [59]:
# 0.0 for no parkin 1.0 for private parking
df['parking'] = df['parking'].fillna(0)

df.loc[df['parking']=='Κλειστό parking','parking']=1.0
df.loc[df['parking']=='Κλειστό parking, Υπόγειο parking','parking']=1.0
df.loc[df['parking']=='Parking πιλωτής','parking']=1.0
df.loc[df['parking']=='Υπόγειο parking','parking']=1.0
df.loc[df['parking']=='Ανοιχτό parking','parking']=0.0
df.loc[df['parking']=='Ανοιχτό parking','parking']=0.0
df.loc[df['parking']=='Parking πιλωτής, Υπόγειο parking','parking']=1.0
df.loc[df['parking']=='Ανοιχτό parking, Κλειστό parking','parking']=0.0
df.loc[df['parking']=='Ανοιχτό parking, Parking πιλωτής','parking']=0.0
df.loc[df['parking']=='Κλειστό parking, Parking πιλωτής','parking']=1.0
df.loc[df['parking']=='Ανοιχτό parking, Κλειστό parking, Parking πιλωτής, Υπόγειο parking','parking']=0.0


df[['parking']] = df[['parking']].apply(pd.to_numeric)

In [165]:
#print(floors('Ισόγειο,1ος,2ος,3ος'))
df['levels'] = df['levels'].fillna(1)

In [166]:
def floors(string):
    totalWords = len(str(string).split(','))
    if totalWords>3:
     return totalWords
    elif string=='Υπερυψωμένο':
        return 0.0
    elif string=='Ημιυπόγειο':
        return 0.0
    elif string=='Υπόγειο':
        return 0.0
    elif string=='Ημιώροφος':
        return 0.0
    elif string=='Ισόγειο':
        return 0.0
    elif string=='Ημιυπόγειο':
        return 0.0
    elif string=='1ος':
        return 1.0
    elif string=='2ος':
        return 2.0
    elif string=='3ος':
        return 3.0
    elif string=='4ος':
        return 4.0
    elif string=='5ος':
        return 5.0
    elif string=='6ος':
        return 6.0
    elif string=='7ος':
        return 7.0        
    elif string=='8ος':
        return 8.0
    else:
       return 100
    

In [167]:
df['levels'] = df['levels'].apply(floors)

In [168]:
#df.energyclass.unique()
#df.levels.unique()
df.levels.dtype


dtype('float64')

In [169]:
def my_function():
    if df['levels'].str.len()>3:
        df['levels'] = df['levels'].str.split(',').str.len()
    elif df.loc[df['levels']=='Υπερυψωμένο','levels']


df.loc[df['levels']=='Υπερυψωμένο','levels']=0.0
df.loc[df['levels']=='Ημιυπόγειο','levels']=0.0
df.loc[df['levels']=='Υπόγειο','levels']=-1.0
df.loc[df['levels']=='Ημιώροφος','levels']=0.0
df.loc[df['levels']=='Ισόγειο','levels']=0.0
df.loc[df['levels']=='Ημιυπόγειο','levels']=-1.0

df.loc[df['levels']=='1ος','levels']=1.0
df.loc[df['levels']=='2ος','levels']=2.0
df.loc[df['levels']=='3ος','levels']=3.0
df.loc[df['levels']=='4ος','levels']=4.0
df.loc[df['levels']=='5ος','levels']=5.0
df.loc[df['levels']=='6ος','levels']=6.0
df.loc[df['levels']=='7ος','levels']=7.0
df.loc[df['levels']=='8ος','levels']=8.0

SyntaxError: expected ':' (2300551958.py, line 4)

In [163]:
df

,location_name,location_region,res_date,res_type,res_address,res_price,res_price_sqr,res_sqr,construction_year,levels,bedrooms,bathrooms,deleted,deleted_at,status,energyclass,auto_heating,solar,cooling,safe_door,gas,fireplace,furniture,student,parking
0,Παλαιό Φάληρο,Αττική,2021-06-18,Διαμέρισμα,"Παλαιό Φάληρο,Γήπεδα",89000,1618.0,55.0,1973.0,0.0,1.0,1.0,1,2022-04-05 00:00:00,Ανακαινισμένο,Δ,1,0,1,1,0,0,0,0,NaN
1,Αχαρνές,Αττική,2022-03-30,Μεζονέτα,"Αχαρνές,Λαθέα",150000,1531.0,98.0,2010.0,4.0,2.0,1.0,1,2022-05-19 00:00:00,Νεόδμητο,Γ,0,0,0,0,0,1,0,0,Κλειστό parking
2,Παγκράτι,Αττική,2022-02-25,Διαμέρισμα,"Αθήνα,Παγκράτι",400000,3175.0,126.0,1990.0,3.0,3.0,1.0,1,2022-06-10 00:00:00,Άριστη,Δ,1,0,1,1,0,1,0,0,NaN
3,Αθήνα,Αττική,2022-03-21,Διαμέρισμα,"Αθήνα,Άγιος Παντελεήμονας",80000,941.0,85.0,1957.0,5.0,2.0,1.0,1,2022-03-23 00:00:00,Καλή,Ζ,1,0,0,1,0,0,0,1,NaN
4,Ίλιον,Αττική,2022-01-11,Διαμέρισμα,"Αθήνα,Άγιος Ελευθέριος",100000,2083.0,48.0,1996.0,1.0,1.0,1.0,1,2022-04-14 00:00:00,Καλή,Δ,1,0,0,1,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Βάρη - Βούλα - Βουλιαγμένη,Αττική,2022-04-10,Διαμέρισμα,"Βούλα,Κάτω Βούλα",145000,2900.0,50.0,NaN,0.0,1.0,1.0,1,2022-05-02 00:00:00,NaN,Εκρεμμεί,0,0,0,0,0,0,0,0,NaN
19996,Αθήνα,Αττική,2022-05-24,Διαμέρισμα,"Αθήνα,Κολιάτσου",92000,1022.0,90.0,1962.0,2.0,3.0,1.0,0,NaN,Ανακαινισμένο,Εκρεμμεί,0,0,0,1,0,0,0,0,NaN
19997,Νίκαια,Αττική,2021-11-24,Κτίριο,"Κορυδαλλός,Όριο Αγίας Βαρβάρας",200000,939.0,213.0,1985.0,0.0,6.0,2.0,0,NaN,NaN,NaN,1,1,1,0,0,0,0,1,NaN
19998,Νέα Ιωνία,Αττική,2022-03-22,Μεζονέτα,"Ηράκλειο,Πράσινος Λόφος",355000,2840.0,125.0,2022.0,0.0,4.0,1.0,1,2022-03-25 00:00:00,Νεόδμητο,Α+,1,1,0,1,0,0,0,0,NaN


                             Address   Long   Lat
0  Jalan Jendral Sodirman 80 Cilacap    NaN   NaN
1  Jalan Jendral Sudirman 80 Cilacap  108.8 -7.49
